# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [93]:
import pandas as pd

data = pd.read_csv('twitter.csv')

data.columns = ['target','id','date','flag','user','text']

data.head(10)

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
9,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


In [94]:
from nltk.tokenize import word_tokenize
import nltk
import ssl
import re
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tom\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [95]:
def clean_up(s):
    
    return re.sub('[^a-z]', ' ', re.sub('http[s]?://[/\w\.]*', '', s.lower())).strip()

In [96]:
def tokenize(s):
    return word_tokenize(s) 

In [97]:
def stem_and_lemmatize(l):
    tokens = [word for word in l if word.isalnum()]
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    [lemmatizer.lemmatize(word) for word in tokens]
    [ps.stem(w) for w in tokens]
    return l

In [98]:
def remove_stopwords(l):
    from nltk.corpus import stopwords
    return list(set(l).difference(stopwords.words()))

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [99]:
data2 = data.sample(5000)

data2 =data2.reset_index()

In [100]:
data2["text_clean"] = data2.text.apply(clean_up)

In [101]:
data2["text_token"] = data2.text_clean.apply(tokenize)

In [102]:
data2["stemlem"] = data2.text_token.apply(stem_and_lemmatize)

In [103]:
data2["text_processed"] = data2.stemlem.apply(remove_stopwords)

In [104]:
data2

,index,target,id,date,flag,user,text,text_clean,text_token,stemlem,text_processed
0,750566,0,2285712853,Mon Jun 22 15:29:02 PDT 2009,NO_QUERY,OakBendReview,"It's so freakin' hot, I can't think of anythin...",it s so freakin hot i can t think of anythin...,"[it, s, so, freakin, hot, i, can, t, think, of...","[it, s, so, freakin, hot, i, can, t, think, of...","[anything, sayyyyyyyyyyy, think, hot, funny, f..."
1,909042,4,1751070359,Sat May 09 18:35:43 PDT 2009,NO_QUERY,DeeLynn,@Medi_Ready What's you're new cite/business ab...,medi ready what s you re new cite business abo...,"[medi, ready, what, s, you, re, new, cite, bus...","[medi, ready, what, s, you, re, new, cite, bus...","[love, hear, business, actually, new, ready, m..."
2,368755,0,2049645675,Fri Jun 05 17:14:44 PDT 2009,NO_QUERY,KellyHillll,i miss kyle already i hope i see him before h...,i miss kyle already i hope i see him before h...,"[i, miss, kyle, already, i, hope, i, see, him,...","[i, miss, kyle, already, i, hope, i, see, him,...","[leaves, hope, stupid, see, miss, already, vac..."
3,529055,0,2195321628,Tue Jun 16 10:58:38 PDT 2009,NO_QUERY,kid_disco,Wish I could remap the Fn and Ctrl keys on my ...,wish i could remap the fn and ctrl keys on my ...,"[wish, i, could, remap, the, fn, and, ctrl, ke...","[wish, i, could, remap, the, fn, and, ctrl, ke...","[fn, ideapad, remap, ctrl, keys, wish, could]"
4,1437675,4,2061214361,Sat Jun 06 20:24:36 PDT 2009,NO_QUERY,AdelyneJazzy,"New moon, can't wait.",new moon can t wait,"[new, moon, can, t, wait]","[new, moon, can, t, wait]","[new, wait, moon]"
...,...,...,...,...,...,...,...,...,...,...,...
4995,1452765,4,2063170397,Sun Jun 07 01:37:32 PDT 2009,NO_QUERY,jenthefangirl,"Thank you to @brokenbacktango, @lijeyeshaveit,...",thank you to brokenbacktango lijeyeshaveit ...,"[thank, you, to, brokenbacktango, lijeyeshavei...","[thank, you, to, brokenbacktango, lijeyeshavei...","[thank, simone, brokenbacktango, leadaisy, lor..."
4996,590622,0,2217387068,Wed Jun 17 20:47:43 PDT 2009,NO_QUERY,krystallouise,"@karawr yeah me too, I've got smokers/sugar te...",karawr yeah me too i ve got smokers sugar tee...,"[karawr, yeah, me, too, i, ve, got, smokers, s...","[karawr, yeah, me, too, i, ve, got, smokers, s...","[yeah, hear, teeth, smokers, got, sugar, karaw..."
4997,295146,0,1996786105,Mon Jun 01 15:41:19 PDT 2009,NO_QUERY,sammlessthan3,didn't get my whole routine done. disappointin...,didn t get my whole routine done disappointin...,"[didn, t, get, my, whole, routine, done, disap...","[didn, t, get, my, whole, routine, done, disap...","[musicians, get, whole, guild, work, routine, ..."
4998,161304,0,1957268633,Fri May 29 00:00:14 PDT 2009,NO_QUERY,traceydukes,"@MissMaryJ M Jizzle I need u to Hit me too, ug...",missmaryj m jizzle i need u to hit me too ugg...,"[missmaryj, m, jizzle, i, need, u, to, hit, me...","[missmaryj, m, jizzle, i, need, u, to, hit, me...","[phone, damn, hit, ugghh, jizzle, seriously, l..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [123]:
bow = data2["text_processed"].tolist()

In [132]:
bow_list = data2['text_processed'].sum()

bow_list

['anything',
 'sayyyyyyyyyyy',
 'think',
 'hot',
 'funny',
 'freakin',
 'love',
 'hear',
 'business',
 'actually',
 'new',
 'ready',
 'medi',
 'leaves',
 'hope',
 'stupid',
 'see',
 'miss',
 'already',
 'vacation',
 'kyle',
 'fn',
 'ideapad',
 'remap',
 'ctrl',
 'keys',
 'wish',
 'could',
 'new',
 'wait',
 'moon',
 'iamjericho',
 'wrestler',
 'dad',
 'mean',
 'nic',
 'bestie',
 'got',
 'time',
 'tonight',
 'spending',
 'fun',
 'tomorrow',
 'home',
 'go',
 'tears',
 'sad',
 'back',
 'loving',
 'broke',
 'silver',
 'afford',
 'jeweler',
 'though',
 'lannagayle',
 'andrewghayes',
 'thank',
 'power',
 'laptop',
 'crying',
 'died',
 'shaking',
 'went',
 'storm',
 'msofarc',
 'club',
 'hope',
 'artfanatic',
 'enjoy',
 'welcome',
 'glad',
 'xox',
 'much',
 'hello',
 'comment',
 'got',
 'ahaha',
 'lauraabee',
 'nafen',
 'known',
 'tonight',
 'driver',
 'workout',
 'slave',
 'french',
 'go',
 'really',
 'trainer',
 'love',
 'got',
 'dismissed',
 'synced',
 'crashs',
 'iphone',
 'syncing',
 'scr

In [137]:
from nltk import FreqDist

bow_freq = FreqDist(bow_list)

bow_freq

FreqDist({'good': 273, 'day': 246, 'like': 234, 'go': 233, 'work': 225, 'get': 220, 'going': 210, 'got': 207, 'today': 207, 'love': 203, ...})

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [159]:
# use the top 5k words in the list
word_features = list(bow_freq.keys())[:5000]

def find_features(row):
    words = row["text_processed"]
    features = {}
    is_positive = (row['target'] != 0)
    for w in word_features:
        features[w] = (w in words)
    
    return features, is_positive


apply_list = data2.apply(find_features, axis=1).to_list()



### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [165]:
training_set = apply_list[:2000]
test_set     = apply_list[3000:]

Accuracy percent 66.64999999999999
Most Informative Features
                     sad = True            False : True   =     23.5 : 1.0
                 awesome = True             True : False  =     13.4 : 1.0
                    made = True             True : False  =     12.8 : 1.0
                     bad = True            False : True   =     10.6 : 1.0
                   check = True             True : False  =      7.9 : 1.0
                 finally = True             True : False  =      7.2 : 1.0
                    lots = True             True : False  =      7.2 : 1.0
                    rock = True             True : False  =      7.2 : 1.0
                    rain = True            False : True   =      6.8 : 1.0
                    year = True            False : True   =      6.8 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [166]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Accuracy percent", (nltk.classify.accuracy(classifier,test_set))*100)

classifier.show_most_informative_features()

Accuracy percent 66.64999999999999
Most Informative Features
                     sad = True            False : True   =     23.5 : 1.0
                 awesome = True             True : False  =     13.4 : 1.0
                    made = True             True : False  =     12.8 : 1.0
                     bad = True            False : True   =     10.6 : 1.0
                   check = True             True : False  =      7.9 : 1.0
                 finally = True             True : False  =      7.2 : 1.0
                    lots = True             True : False  =      7.2 : 1.0
                    rock = True             True : False  =      7.2 : 1.0
                    rain = True            False : True   =      6.8 : 1.0
                    year = True            False : True   =      6.8 : 1.0


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here